In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# --- 1. Dados de Treinamento ---
# Características: [Peso (gramas), Cor (escala 1 a 10)]
# Rótulos: 0 para Maçã, 1 para Banana

# Fruta A: Maçã (150g, Cor 8)
# Fruta B: Maçã (160g, Cor 9)
# Fruta C: Maçã (140g, Cor 7)
# Fruta D: Banana (100g, Cor 2)
# Fruta E: Banana (110g, Cor 3)
# Fruta F: Banana (90g, Cor 1)
dados_treinamento = np.array([
    [150, 8],
    [160, 9],
    [140, 7],
    [100, 2],
    [110, 3],
    [90, 1]
])

# Rótulos correspondentes às frutas (0 = Maçã, 1 = Banana)
rotulos_treinamento = np.array([0, 0, 0, 1, 1, 1])

# --- Opcional, mas Recomendado: Escalonamento dos Dados ---
# É importante que as features (peso e cor) estejam na mesma escala
# para que uma não domine a outra na cálculo da distância.
scaler = StandardScaler()
dados_treinamento_escalado = scaler.fit_transform(dados_treinamento)

# --- 2. Criar e Treinar o Modelo KNN ---
# Vamos usar K = 3, como no exemplo manual
k = 4
knn_model = KNeighborsClassifier(n_neighbors=k)

# O treinamento no KNN é basicamente "memorizar" os dados
knn_model.fit(dados_treinamento_escalado, rotulos_treinamento)

# --- 3. Nova Fruta para Classificar ---
# Nova Fruta: Peso 130g, Cor 6
nova_fruta = np.array([[130, 6]])

# Precisamos escalar a nova fruta usando o MESMO scaler que treinamos
nova_fruta_escalada = scaler.transform(nova_fruta)

# --- 4. Fazer a Previsão ---
previsao = knn_model.predict(nova_fruta_escalada)

# Obter as probabilidades para cada classe (opcional)
probabilidades = knn_model.predict_proba(nova_fruta_escalada)

# Mapear o resultado numérico para o nome da fruta
mapeamento_rotulos = {0: "Maçã", 1: "Banana"}
resultado_final = mapeamento_rotulos[previsao[0]]

# --- 5. Mostrar os Resultados ---
print(f"Características da Nova Fruta (Peso, Cor): {nova_fruta[0]}")
print(f"A Nova Fruta foi classificada como: **{resultado_final}**")
print(f"Probabilidades por classe (Maçã, Banana): {probabilidades[0]}")

# Para entender quem são os vizinhos:
# knn_model.kneighbors retorna as distâncias e os índices dos vizinhos
distancias, indices = knn_model.kneighbors(nova_fruta_escalada)

print(f"\nOs {k} vizinhos mais próximos são:")
for i in range(k):
    indice_vizinho = indices[0][i]
    distancia_vizinho = distancias[0][i]
    caracteristicas_vizinho = dados_treinamento[indice_vizinho]
    rotulo_vizinho = mapeamento_rotulos[rotulos_treinamento[indice_vizinho]]
    print(f"  - Fruta {chr(65 + indice_vizinho)}: {caracteristicas_vizinho} (Categoria: {rotulo_vizinho}, Distância: {distancia_vizinho:.2f})")

# Para confirmar a contagem de votos (baseado nos vizinhos que encontramos)
vizinhos_encontrados_rotulos = [rotulos_treinamento[idx] for idx in indices[0]]
contagem_votos = {mapeamento_rotulos[label]: vizinhos_encontrados_rotulos.count(label) for label in set(vizinhos_encontrados_rotulos)}
print(f"\nContagem de votos dos {k} vizinhos: {contagem_votos}")

Características da Nova Fruta (Peso, Cor): [130   6]
A Nova Fruta foi classificada como: **Maçã**
Probabilidades por classe (Maçã, Banana): [0.75 0.25]

Os 4 vizinhos mais próximos são:
  - Fruta C: [140   7] (Categoria: Maçã, Distância: 0.50)
  - Fruta A: [150   8] (Categoria: Maçã, Distância: 1.00)
  - Fruta E: [110   3] (Categoria: Banana, Distância: 1.23)
  - Fruta B: [160   9] (Categoria: Maçã, Distância: 1.49)

Contagem de votos dos 4 vizinhos: {'Maçã': 3, 'Banana': 1}
